In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from reproject import reproject_from_healpix, reproject_to_healpix
from astropy_healpix import HEALPix
from astropy.coordinates import Galactic
from astropy.coordinates import SkyCoord
import healpy
import gc

### Choose type of starting file

In [ ]:
#############################
start_with_healpix = True # True if already reading in Healpix files
#############################

### Read in Q, U data in equatorial

In [ ]:
hduQ = fits.open('/srv/data/dva/dragons_hoard/Data_cubes_Jun_2025_BB/convolved_masked_cubes/500to1000/conv_hpx/q_499to1000MHz_convolved_2.5deg_masked_eq.hpx.fits')
hdrQ = hduQ[0].header
Qeq = hduQ[0].data

hduU = fits.open('/srv/data/dva/dragons_hoard/Data_cubes_Jun_2025_BB/convolved_masked_cubes/500to1000/conv_hpx/u_499to1000MHz_convolved_2.5deg_masked_eq.hpx.fits')
hdrU = hduU[0].header
Ueq = hduU[0].data

hduI = fits.open('/srv/data/dva/dragons_hoard/Data_cubes_Jun_2025_BB/convolved_masked_cubes/500to1000/conv_hpx/i_499to1000MHz_convolved_2.5deg_masked_eq.hpx.fits')
hdrI = hduI[0].header
Ieq = hduI[0].data

print(Qeq.shape)

print(repr(hdrQ))

### Set nside for Healpix depending on pixel sizes

In [ ]:
if start_with_healpix == False:
    
    dx = hdrQ['CDELT2']
    
    if dx >= 1.0:
        nside = 64
    if ((dx < 1.0) & (dx >= 0.5)):
        nside = 128
    if ((dx < 0.5) & (dx >= 0.25)):
        nside = 256
    if ((dx < 0.25) & (dx >= 0.125)):
        nside = 512
        
    if dx < 0.125:
        print('WARNING: really small pixels!')
    else:
        print('Healpix using nside = '+str(nside))

    numfreq = hdrQ['NAXIS3']

else:
    numfreq = hdrQ['NAXIS2']
    nside = hdrQ['NSIDE']

print('channels: ',numfreq)
print('nside: ',nside)

### Make 2D header

In [ ]:
if start_with_healpix == False:
    wcs2D = WCS(hdrQ).dropaxis(1)
    print(repr(wcs2D))

### Make sure invalid data set to NaN

In [ ]:
# In case there are stray UNSEENs in the input files
Qeq[Qeq == healpy.UNSEEN] = np.nan
Ueq[Ueq == healpy.UNSEEN] = np.nan
Ieq[Ieq == healpy.UNSEEN] = np.nan

Qeq[~np.isfinite(Qeq)] = np.nan
Ueq[~np.isfinite(Ueq)] = np.nan
Ieq[~np.isfinite(Ieq)] = np.nan

Qeq[Qeq == 0] = np.nan
Ueq[Ueq == 0] = np.nan
Ieq[Ieq == 0] = np.nan


### Reproject to healpix equatorial

In [ ]:
if start_with_healpix:
    print('already have healpix file')
    Qeq_hpx = Qeq
    Ueq_hpx = Ueq
    Ieq_hpx = Ieq

else:
    Nhpx = healpy.pixelfunc.nside2npix(nside)

    Qeq_hpx = np.empty([hdrQ['NAXIS3'],Nhpx])
    Ueq_hpx = np.empty([hdrU['NAXIS3'],Nhpx])
    Ieq_hpx = np.empty([hdrI['NAXIS3'],Nhpx])

    for i in range(0,hdrQ['NAXIS3']):
        print('Converting to Healpix '+str(i+1)+' of '+str(hdrQ['NAXIS3']))
        Ueq_hpx[i], footprint = reproject_to_healpix((Ueq[i],wcs2D),coord_system_out='C',nside=nside)
        Qeq_hpx[i], footprint = reproject_to_healpix((Qeq[i],wcs2D),coord_system_out='C',nside=nside)
        Ieq_hpx[i], footprint = reproject_to_healpix((Ieq[i],wcs2D),coord_system_out='C',nside=nside)

### Delete original (non-healpix) cubes to reduce memory usage:

In [ ]:
if start_with_healpix == False:
    del Qeq
    del Ueq
    del Ieq
    
    gc.collect()

In [ ]:
healpy.mollview(Qeq_hpx[0],min=-20,max=20,rot=(0,90),cmap='RdBu_r')
healpy.mollview(Ueq_hpx[0],min=-20,max=20,rot=(0,90),cmap='RdBu_r')
healpy.mollview(Ieq_hpx[0],min=0,max=1000,rot=(0,90))

### Change U from IAU to COSMO convention and apply Healpy masking to NaNs
### WARNING: DO NOT ACCIDENTALLY RUN THIS CELL TWICE!

In [ ]:
Ueq_hpx *= -1

Ieq_hpx[np.isnan(Ieq_hpx)] = healpy.UNSEEN
Qeq_hpx[np.isnan(Qeq_hpx)] = healpy.UNSEEN
Ueq_hpx[np.isnan(Ueq_hpx)] = healpy.UNSEEN


### Rotate to Galactic

In [ ]:
r = healpy.Rotator(coord=['C', 'G'])

if start_with_healpix:
    ii = hdrQ['NAXIS2']
    Qgal_hpx = np.empty_like(Qeq)
    Ugal_hpx = np.empty_like(Ueq)
    Igal_hpx = np.empty_like(Ieq)
else:
    ii  = hdrQ['NAXIS3']
    Qgal_hpx = np.empty_like(Qeq_hpx)
    Ugal_hpx = np.empty_like(Ueq_hpx)
    Igal_hpx = np.empty_like(Ieq_hpx)

for i in range(0,ii):
    print('Rotating to Galactic '+str(i+1)+' of '+str(numfreq))

    Igal_hpx[i], Qgal_hpx[i], Ugal_hpx[i] = r.rotate_map_alms([Ieq_hpx[i],Qeq_hpx[i],Ueq_hpx[i]])
    i_rot_pix,   q_rot_pix,   u_rot_pix   = r.rotate_map_pixel([Ieq_hpx[i],Qeq_hpx[i],Ueq_hpx[i]])

    # Getting rid of bad values:
    Igal_hpx[i][(i_rot_pix < -1e20) | (i_rot_pix > 1e20)] = healpy.UNSEEN
    Qgal_hpx[i][(q_rot_pix < -1e20) | (q_rot_pix > 1e20)] = healpy.UNSEEN
    Ugal_hpx[i][(u_rot_pix < -1e20) | (u_rot_pix > 1e20)] = healpy.UNSEEN

### Apply NaNs back to masking and change U back to IAU
### WARNING: DO NOT ACCIDENTALLY RUN THIS CELL TWICE!

In [ ]:
Qgal_hpx = np.array(Qgal_hpx)
Ugal_hpx = np.array(Ugal_hpx)
Igal_hpx = np.array(Igal_hpx)

Qgal_hpx[Qgal_hpx == healpy.UNSEEN] = np.nan
Ugal_hpx[Ugal_hpx == healpy.UNSEEN] = np.nan
Igal_hpx[Igal_hpx == healpy.UNSEEN] = np.nan

Ugal_hpx *= -1

In [ ]:
healpy.mollview(Qgal_hpx[0],min=-20,max=20,cmap='RdBu_r',rot=(0,0))

### Make new Galactic header (plate caree)

In [ ]:
pixsize = 0.5

nx = int(round((360/pixsize),0))
ny = int(round((180/pixsize),0))

hdr_gal = hdrQ.copy()

if start_with_healpix:
    hdr_gal['NAXIS3']  = hdrQ['NAXIS2']
    hdr_gal['NAXIS']  = 3
    hdr_gal['CTYPE3'] = hdrQ['CTYPE2']
    hdr_gal['CRPIX3'] = hdrQ['CRPIX2']
    hdr_gal['CRVAL3'] = hdrQ['CRVAL2']
    hdr_gal['CDELT3'] = hdrQ['CDELT2']
    hdr_gal['CUNIT3'] = hdrQ['CUNIT2']

    try:
        del hdr_gal['PIXTYPE']
        del hdr_gal['ORDERING']
        del hdr_gal['INDXSCHM']
        del hdr_gal['OBJECT']
        del hdr_gal['NSIDE']
    except:
        pass

hdr_gal['COORDSYS']  = 'galactic'
hdr_gal['NAXIS1']  = nx 
hdr_gal['NAXIS2']  = ny

hdr_gal['CTYPE1']  = 'GLON-CAR'
hdr_gal['CRPIX1']  = nx/2.+0.5 
hdr_gal['CRVAL1']  = 0          
hdr_gal['CDELT1']  = -pixsize
hdr_gal['CUNIT1']  = 'deg'

hdr_gal['CTYPE2']  = 'GLAT-CAR'
hdr_gal['CRPIX2']  = ny/2.+0.5
hdr_gal['CRVAL2']  = 0
hdr_gal['CDELT2']  = pixsize
hdr_gal['CUNIT2']  = 'deg'
        
print(repr(hdr_gal))
print('=============================')

wcs2D_gal = WCS(hdr_gal).dropaxis(2)
print(repr(wcs2D_gal))


### Make new Galactic header (healpix)

In [ ]:
hdr_gal_hpx = hdrQ.copy()

hdr_gal_hpx['COORDSYS']  = 'galactic'

print(repr(hdr_gal_hpx))
print('=============================')

wcs2D_gal_hpx = WCS(hdr_gal_hpx).dropaxis(1)
print(repr(wcs2D_gal_hpx))


### Convert Galactic healpix to regular FITS

In [ ]:
Q_gal = np.empty([hdr_gal['NAXIS3'],hdr_gal['NAXIS2'],hdr_gal['NAXIS1']])
U_gal = np.empty([hdr_gal['NAXIS3'],hdr_gal['NAXIS2'],hdr_gal['NAXIS1']])
I_gal = np.empty([hdr_gal['NAXIS3'],hdr_gal['NAXIS2'],hdr_gal['NAXIS1']])

for i in range(0,Q_gal.shape[0]):
#for i in range(0,10):

    print('Converting back to FITS plate caree '+str(i+1)+' of '+str(numfreq))
    
    Q_gal[i,:,:], footprint = reproject_from_healpix((Qgal_hpx[i],'galactic'),wcs2D_gal, nested=False)
    U_gal[i,:,:], footprint = reproject_from_healpix((Ugal_hpx[i],'galactic'),wcs2D_gal, nested=False)
    I_gal[i,:,:], footprint = reproject_from_healpix((Igal_hpx[i],'galactic'),wcs2D_gal, nested=False)
    

## Write out files

In [ ]:
fits.writeto('/srv/data/dva/dragons_hoard/Data_cubes_Jun_2025_BB/convolved_masked_cubes/500to1000/conv_hpx/q_499to1000MHz_convolved_2.5deg_masked_gal.hpx.fits', 
             Qgal_hpx, hdr_gal_hpx, overwrite=False)

fits.writeto('/srv/data/dva/dragons_hoard/Data_cubes_Jun_2025_BB/convolved_masked_cubes/500to1000/conv_hpx/u_499to1000MHz_convolved_2.5deg_masked_gal.hpx.fits', 
             Ugal_hpx, hdr_gal_hpx, overwrite=False)

fits.writeto('/srv/data/dva/dragons_hoard/Data_cubes_Jun_2025_BB/convolved_masked_cubes/500to1000/conv_hpx/i_499to1000MHz_convolved_2.5deg_masked_gal.hpx.fits', 
             Igal_hpx, hdr_gal_hpx, overwrite=False)


In [ ]:
fits.writeto('/srv/data/dva/dragons_hoard/Data_cubes_Jun_2025_BB/convolved_masked_cubes/500to1000/conv_plat_car/q_499to1000MHz_convolved_2.5deg_masked_gal.fits', 
             Q_gal, hdr_gal, overwrite=False)

fits.writeto('/srv/data/dva/dragons_hoard/Data_cubes_Jun_2025_BB/convolved_masked_cubes/500to1000/conv_plat_car/u_499to1000MHz_convolved_2.5deg_masked_gal.fits', 
             U_gal, hdr_gal, overwrite=False)

fits.writeto('/srv/data/dva/dragons_hoard/Data_cubes_Jun_2025_BB/convolved_masked_cubes/500to1000/conv_plat_car/i_499to1000MHz_convolved_2.5deg_masked_gal.fits', 
             I_gal, hdr_gal, overwrite=False)
